In [192]:
import numpy as np
from scipy import stats

In [2]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.2 MB/s eta 0:00:00


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle

from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC

In [59]:
from sklearn.metrics import accuracy_score

In [5]:
df = pd.read_csv('/content/dataset.tsv', sep="\t", encoding="utf-8")

In [6]:
df.head()

,page_id,text,label,Pattern Category
0,1012,FLASH SALE | LIMITED TIME ONLY Shop Now,1,Urgency
1,158,Pillowcases & Shams,0,Not Dark Pattern
2,108,Write a review,0,Not Dark Pattern
3,1425,"To start your return, simply click on the foll...",0,Not Dark Pattern
4,1658,newsletter signup (privacy policy),0,Not Dark Pattern


In [35]:
map = {j:i for i,j in  enumerate(df['Pattern Category'].unique())}

In [38]:
map

{'Urgency': 0,
 'Not Dark Pattern': 1,
 'Scarcity': 2,
 'Misdirection': 3,
 'Social Proof': 4,
 'Obstruction': 5,
 'Sneaking': 6,
 'Forced Action': 7}

In [11]:
df1 = df.drop(columns = ['page_id','label'])

In [40]:
X_train,X_test,Y_train,Y_test  = train_test_split(df1['text'],df1['Pattern Category'],test_size=0.3)

In [226]:
vect = TfidfVectorizer(max_features = 100,lowercase=True)

In [98]:
Y_train

1992    Not Dark Pattern
1576        Misdirection
2064    Not Dark Pattern
275     Not Dark Pattern
2007            Scarcity
              ...       
1249        Social Proof
23          Social Proof
1941        Social Proof
1744    Not Dark Pattern
2020    Not Dark Pattern
Name: Pattern Category, Length: 1649, dtype: object

In [99]:
X_train

1992       PopWallet+ Refine by Product Type: PopWallet+ 
1576                  No, thanks. I'd prefer not to save.
2064                                         Sports Bras 
275                                    Parad Combo Deals 
2007                                        Only 69 left!
                              ...                        
1249    21 people looked at this during the last 24 hours
23        Michael in Pontiac, United States purchased a  
1941                                 89 currently viewing
1744    Like many websites, we share certain informati...
2020                 Submitted by Fermin Larracilla Pérez
Name: text, Length: 1649, dtype: object

In [100]:
X_test

165                           IN 43 PEOPLE'S SHOPPING BAG
1529                            Only 1 unit left in stock
497     Someone from Airdrie & 10820 others bought thi...
148             Your order is reserved for 08:48 minutes!
1582     This product can expose you to chemicals incl...
                              ...                        
761                         Almost Sold Out! Only 2 Left.
1072                            Only 1 Left in this Size!
1793                           Port, Sherry & Sweet Wine 
2142                                     Quick Navigation
2113                Floral Cross Stitch & Embroidery Kits
Name: text, Length: 707, dtype: object

In [101]:
Y_test

165         Social Proof
1529            Scarcity
497         Social Proof
148              Urgency
1582    Not Dark Pattern
              ...       
761             Scarcity
1072            Scarcity
1793    Not Dark Pattern
2142    Not Dark Pattern
2113    Not Dark Pattern
Name: Pattern Category, Length: 707, dtype: object

In [227]:
x_train = vect.fit_transform(X_train)

In [228]:
y_train = Y_train.map(map)

In [229]:
rfc = RandomForestClassifier()

In [230]:
rfc.fit(x_train,y_train)

RandomForestClassifier()

In [231]:
x_test = vect.transform(X_test)

In [232]:
preds = rfc.predict(x_test)

In [233]:
accuracy_score(preds,Y_test.map(map))

0.9080622347949081

In [240]:
cbc = CatBoostClassifier()
cbc.fit(x_train,y_train)

Learning rate set to 0.081171
0:	learn: 1.8411277	total: 36.7ms	remaining: 36.7s
1:	learn: 1.6535756	total: 62.1ms	remaining: 31s
2:	learn: 1.5276493	total: 87ms	remaining: 28.9s
3:	learn: 1.4236719	total: 114ms	remaining: 28.3s
4:	learn: 1.3248466	total: 139ms	remaining: 27.6s
5:	learn: 1.2466413	total: 164ms	remaining: 27.1s
6:	learn: 1.1793517	total: 191ms	remaining: 27.1s
7:	learn: 1.1179837	total: 220ms	remaining: 27.3s
8:	learn: 1.0612125	total: 246ms	remaining: 27s
9:	learn: 1.0157455	total: 271ms	remaining: 26.8s
10:	learn: 0.9737300	total: 296ms	remaining: 26.6s
11:	learn: 0.9377174	total: 321ms	remaining: 26.4s
12:	learn: 0.9040107	total: 347ms	remaining: 26.3s
13:	learn: 0.8732059	total: 372ms	remaining: 26.2s
14:	learn: 0.8480642	total: 397ms	remaining: 26.1s
15:	learn: 0.8203060	total: 425ms	remaining: 26.2s
16:	learn: 0.7959534	total: 450ms	remaining: 26s
17:	learn: 0.7782618	total: 474ms	remaining: 25.9s
18:	learn: 0.7605181	total: 502ms	remaining: 25.9s
19:	learn: 0.741

In [235]:
cbc_preds = cbc.predict(x_test)

In [236]:
accuracy_score(cbc_preds,Y_test.map(map))

0.4724186704384724

In [237]:
svc = SVC()
svc.fit(x_train,y_train)
preds_svc = svc.predict(x_test)

accuracy_score(preds_svc,Y_test.map(map))


0.9052333804809052

In [238]:
def ensemble(text):
  text = [i.lower() for i in text]
  text = vect.transform(text)

  results = [rfc.predict(text)[0],svc.predict(text)[0],cbc.predict(text)[0][0]]

  return reverse_map[stats.mode(results).mode]

In [176]:
map

{'Urgency': 0,
 'Not Dark Pattern': 1,
 'Scarcity': 2,
 'Misdirection': 3,
 'Social Proof': 4,
 'Obstruction': 5,
 'Sneaking': 6,
 'Forced Action': 7}

In [148]:
reverse_map

{0: 'Urgency',
 1: 'Not Dark Pattern',
 2: 'Scarcity',
 3: 'Misdirection',
 4: 'Social Proof',
 5: 'Obstruction',
 6: 'Sneaking',
 7: 'Forced Action'}

In [189]:
reverse_map = {i:j for i,j in  enumerate(df['Pattern Category'].unique())}

In [216]:
testing2 = ['This Nespresso coffee assortment offers 30 Nespresso Pods in a variety of 3 best-selling coffee blends for the Nespresso VertuoLine System.10 Stormio, 10 Odacio, 10 Melozio']

In [239]:
ensemble(testing2)

'Not Dark Pattern'